In [ ]:
import os
from pathlib import Path
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine

from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [ ]:
def search_costo_ventas_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    param_dia_hr_ini = pd.Timestamp(2024, mes, 1) + pd.Timedelta(hours=7)
    param_dia_hr_fin = pd.Timestamp(2024, mes+1, 1) + pd.Timedelta(hours=7) - pd.Timedelta(seconds= 1)
    
    search_costo_ventas = [
        "&", "&",
            ("state", "in", ["purchase", "done"]),
            ("date_approve", ">=", param_dia_hr_ini.strftime('%Y-%m-%d %H:%M:%S')),
            ("date_approve", "<=", param_dia_hr_fin.strftime('%Y-%m-%d %H:%M:%S')),
        ]

    return search_costo_ventas

In [ ]:
def get_dfs_from_database(db_mode: str, mes:int) -> list[pd.DataFrame]:
    
    if db_mode.lower() == 'local':
    
        db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/comisiones.db'))
        engine = create_engine(f'sqlite:///{db_file_path_str}')

        with engine.connect() as conn, conn.begin():
            ventas = pd.read_sql_table(f'ventas_{mes}_2024', conn, dtype_backend='numpy_nullable')
            try:
                ultimo_costo = pd.read_sql_table(f'ultimo_costo_{mes}_2024', conn, dtype_backend='numpy_nullable')
            except:
                ultimo_costo = None

        engine.dispose()
    

    elif db_mode.lower() != 'local':
        raise Exception (f'Sólo existe la base de datos "Local"')


    return ventas, ultimo_costo

In [ ]:
def _get_df_from_excel(file_name:str, file_sheet:str, file_location:str) -> pd.DataFrame:

    file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{file_location}/{file_name}'))
    df = pd.read_excel(file_path_str, sheet_name=file_sheet, dtype_backend='numpy_nullable')

    return df

In [ ]:
def prov_locales_df_from_excel() -> list[pd.DataFrame]:
    
    file_name = 'proveedores_oficiales.xlsx'
    file_sheet = 'Hoja1'
    file_location = 'compras'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    prov_oficiales = df.loc[df['oficial'] == 1][['partner_id', 'partner_name']]
    prov_locales = df.loc[df['oficial'] == 0][['partner_id', 'partner_name']]

    return prov_oficiales, prov_locales

In [ ]:
def ultimo_costo_sae_df_from_excel() -> list[pd.DataFrame]:
    
    file_name = 'ultimo_costo_sae.xlsx'
    file_sheet = 'Hoja1'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    return df

In [ ]:
def api_call_purchase_doc_func(api_params: dict, search_costo_ventas: list[str] ) -> list[int, dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_doc_fields = [
                    'name',
                    'state',
                    'partner_id',
                    'partner_ref',
                    'date_approve',
                    'x_fecha_factura',
                    'user_id',
                    'create_uid'
                    ]

    purchase_doc_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [search_costo_ventas])
    purchase_doc_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [purchase_doc_ids], {'fields': purchase_doc_fields})
    
    return purchase_doc_ids, purchase_doc_json

In [ ]:
def purchase_doc_func(purchase_doc_json: list[dict], prov_oficiales:pd.DataFrame) -> pd.DataFrame:

    purchase_doc_data = []

    for compra in purchase_doc_json:
        new = {}
        new['order_id'] = compra['id']
        new['order_name'] = compra['name']
        new['order_state'] = compra['state']
        new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
        new['partner_id'] = compra['partner_id'][0]
        new['partner_name'] = compra['partner_id'][1]
        new['partner_fact_ref'] = compra['partner_ref']
        new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
        new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
        new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

        purchase_doc_data.append(new)

    compras_doc = pd.DataFrame(purchase_doc_data)
    compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
    compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')
    compras_doc['oficial'] = compras_doc['partner_id'].isin(prov_oficiales['partner_id'])

    return compras_doc


In [ ]:
def api_call_purchase_line_func(api_params: dict, purchase_doc_ids: list[int]) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_line_fields = [
                        'order_id',
                        'date_approve',
                        'partner_id',
                        'product_id',
                        'product_qty',
                        'price_unit_discounted'
                        ]

    purchase_line_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", purchase_doc_ids)]])
    purchase_line_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_line_ids], {'fields': purchase_line_fields})
    
    return purchase_line_json

In [ ]:
def purchase_line_func(purchase_line_json: list[dict]) -> pd.DataFrame:
    
    purchase_line_data = []

    for line in purchase_line_json:
        new = {}
        new['line_id'] = line['id']
        new['order_id'] = line['order_id'][0]
        new['order_name'] = line['order_id'][1]
        new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
        new['partner_id'] = line['partner_id'][0]
        new['partner_name'] = line['partner_id'][1]
        new['product_id_pp'] = line['product_id'][0]
        new['product_name'] = line['product_id'][1]
        new['product_qty'] = line['product_qty']
        new['product_cost'] = line['price_unit_discounted']
        
        purchase_line_data.append(new)

    compras_line = pd.DataFrame(purchase_line_data)
    compras_line['order_date'] = pd.to_datetime(compras_line['order_date'], format='%Y-%m-%d %H:%M:%S')

    return compras_line

In [ ]:
def compras_func(compras_doc:pd.DataFrame, compras_line:pd.DataFrame) -> pd.DataFrame:
    
    compras_odoo = pd.merge(
                    compras_line,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

    compras_odoo['order_date'] = compras_odoo['order_date'].dt.normalize()

    cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
    compras_odoo[cols_to_Int64] = compras_odoo[cols_to_Int64].astype('Int64')

    compras_odoo['product_qty'] = compras_odoo['product_qty'].astype('Float64')
    compras_odoo['vendedora'] = compras_odoo['vendedora'].convert_dtypes()

    return compras_odoo

# Pruebas

In [ ]:
mes = 1
db_mode = 'local'

api_params = api_params_func()
search_costo_ventas = search_costo_ventas_func(mes)

ventas, ultimo_costo = get_dfs_from_database(db_mode, mes)

if not ultimo_costo:
    ultimo_costo = ultimo_costo_sae_df_from_excel()

prov_oficiales, prov_locales = prov_locales_df_from_excel()

purchase_doc_ids, purchase_doc_json = api_call_purchase_doc_func(api_params, search_costo_ventas)
purchase_line_json = api_call_purchase_line_func(api_params, purchase_doc_ids)

compras_doc = purchase_doc_func(purchase_doc_json, prov_oficiales)
compras_line = purchase_line_func(purchase_line_json)
compras_odoo = compras_func(compras_doc, compras_line)


# Desarrollo del código

In [ ]:
lista_capturistas = [
       'Elsa Ivette Diaz Leyva',
       'Alexa Yadira Mazariegos Zunun',
       'Dulce Guadalupe Pedroza Valenzuela',
       'Mariana Araceli Carvajal Flores',
       'Rosario Martinez Zarate'
]

# Compras especiales que además no son de proveedores oficiales
compras_especiales = compras_odoo.loc[
            (~compras_odoo['vendedora'].isin(lista_capturistas))
            & (~compras_odoo['partner_id'].isin(prov_oficiales['partner_id']))
       ]

# Resto de compras, es decir, compras de venta normal
compras_no_especiales = compras_odoo.loc[
              ~compras_odoo['line_id'].isin(compras_especiales['line_id'])
       ]

In [ ]:
match_1_repetidos = pd.DataFrame([], columns=['line_id','fact_line_id'])

compras_especiales_while = compras_especiales.copy()
ventas_while = ventas.copy()
repetidos_while = True

while repetidos_while:
    
    match_1_merge = (
        pd.merge_asof(
            left = compras_especiales_while.sort_values('order_date'), 
            right = ventas_while.sort_values('invoice_date'),
            
            left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
            right_by = ['product_id', 'salesperson_name', 'quantity'], 
            
            left_on = 'order_date', 
            right_on = 'invoice_date', 

            direction = 'nearest',
            tolerance = pd.Timedelta(days=3))
    )

    ids_ventas_repetidas_por_corregir = match_1_merge.loc[
                                                (~match_1_merge['fact_line_id'].isna()) 
                                                & (match_1_merge['fact_line_id'].duplicated())
                                        ]['fact_line_id']
    
    if not ids_ventas_repetidas_por_corregir.empty:
        repetidos_mini_df = match_1_merge.loc[match_1_merge['fact_line_id'].isin(ids_ventas_repetidas_por_corregir)].sort_values('line_id')
        repetidos_mini_df['diff'] = abs(repetidos_mini_df['invoice_date'] - repetidos_mini_df['order_date'])

        for id in ids_ventas_repetidas_por_corregir:
            match_line_to_keep = repetidos_mini_df.loc[repetidos_mini_df['fact_line_id'] == id].sort_values('diff').reset_index().loc[:0, ['line_id', 'fact_line_id']]

            if match_1_repetidos.empty:
                match_1_repetidos = match_line_to_keep.copy()
            else:
                match_1_repetidos = pd.concat([match_1_repetidos, match_line_to_keep])

        compras_especiales_while = compras_especiales_while[~compras_especiales_while['line_id'].isin(match_1_repetidos['line_id'])]
        ventas_while = ventas_while[~ventas_while['fact_line_id'].isin(match_1_repetidos['fact_line_id'])]


    if ids_ventas_repetidas_por_corregir.empty:        
        print('Terminó el ciclo while')
        repetidos_while = False


In [ ]:
# Match resultantes del 1er match.
match_1 = match_1_merge.loc[~match_1_merge['fact_line_id'].isna(), ['line_id', 'fact_line_id']]

# Después de correr el 1er match, las ventas restantes
ventas_after_match_1 = ventas.loc[
                                (~ventas['fact_line_id'].isin(match_1['fact_line_id']))
                                & (~ventas['fact_line_id'].isin(match_1_repetidos['fact_line_id']))
                            ]

# Después de correr el 1er match, las compras que tienen un product_id que sí existe en las ventas restantes.
compras_especiales_after_match_1 = compras_especiales.loc[
                                            (~compras_especiales['line_id'].isin(match_1['line_id']))
                                            & (~compras_especiales['line_id'].isin(match_1_repetidos['line_id']))
                                            & (compras_especiales['product_id_pp'].isin(ventas_after_match_1['product_id']))
                                        ]

# Después de correr el 1er match, resto de las compras especiales. Estan no tienen un product_id que existe en las ventas restantes y no se pueden merchar.
compras_especiales_after_match_1_sin_linea_venta = compras_especiales.loc[
                                                            (~compras_especiales['line_id'].isin(match_1['line_id']))
                                                            & (~compras_especiales['line_id'].isin(match_1_repetidos['line_id']))
                                                            & (~compras_especiales['product_id_pp'].isin(ventas_after_match_1['product_id']))
                                                        ]

In [ ]:
# Varias ventas para una sóla compra.
match_2 = pd.DataFrame([], columns=['line_id','fact_line_id'])

for i in range(len(compras_especiales_after_match_1)):
    
    linea_compra = compras_especiales_after_match_1.sort_values('order_date').iloc[i]

    mini_df = ventas_after_match_1.loc[
                (~ventas_after_match_1['fact_line_id'].isin(match_2['fact_line_id']))
                & (ventas_after_match_1['salesperson_name'] == linea_compra['vendedora'])
                & (ventas_after_match_1['product_id'] == linea_compra['product_id_pp'])
            ]
    
    if not mini_df.empty:
        df_copia = mini_df.copy()
        df_copia['diff'] = abs(df_copia['invoice_date'] - linea_compra['order_date'])
        df_sort = df_copia.sort_values('diff').reset_index()
        df_sort['cumsum'] = df_sort['quantity'].cumsum()
        index = df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index[0] if not df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index.empty else None
        if index != None:
            df_sort.loc[:index , 'line_id'] = linea_compra['line_id']
            df_to_keep = df_sort.loc[~df_sort['line_id'].isna()]

            if match_2.empty:
                match_2 = df_to_keep[['line_id','fact_line_id']].copy()
            else:
                match_2 = pd.concat([
                            match_2, 
                            df_to_keep[['line_id','fact_line_id']],
                        ])

In [ ]:
# Después de correr el 2do match, las ventas restantes
ventas_after_match_2 = ventas_after_match_1.loc[
                                ~ventas_after_match_1['fact_line_id'].isin(match_2['fact_line_id'])
                            ]

# Después de correr el 2do match, las compras que restan y que sí tendrán venta a la cual mercharse.
compras_especiales_after_match_2 = compras_especiales_after_match_1.loc[
                                            (~compras_especiales_after_match_1['line_id'].isin(match_2['line_id']))
                                            & (compras_especiales_after_match_1['product_id_pp'].isin(ventas_after_match_2['product_id']))
                                        ]

# Después de correr el 2do match, resto de las compras especiales. Estan no tienen un product_id que existe en las ventas restantes y no se pueden merchar.
compras_especiales_after_match_2_sin_linea_venta = compras_especiales_after_match_1.loc[
                                                            (~compras_especiales_after_match_1['line_id'].isin(match_2['line_id']))
                                                            & (~compras_especiales_after_match_1['product_id_pp'].isin(ventas_after_match_2['product_id']))
                                                        ]

In [ ]:
# Varias compras para una sóla venta.
match_3 = pd.DataFrame([], columns=['line_id','fact_line_id'])
match_3_to_errase = pd.DataFrame([], columns=['line_id','fact_line_id'])

ventas_inside_compras_especiales_after_match_2 = ventas_after_match_2[
                                                        ventas_after_match_2['product_id'].isin(compras_especiales_after_match_2['product_id_pp'])
                                                    ]

for i in range(len(ventas_inside_compras_especiales_after_match_2)):
    
    linea_venta = ventas_inside_compras_especiales_after_match_2.sort_values('invoice_date').iloc[i]

    mini_df = compras_especiales_after_match_2.loc[
                (~compras_especiales_after_match_2['line_id'].isin(match_3['line_id']))
                & (compras_especiales_after_match_2['vendedora'] == linea_venta['salesperson_name'])
                & (compras_especiales_after_match_2['product_id_pp'] == linea_venta['product_id'])
            ]
        

    if not mini_df.empty:
    
        df_copia = mini_df.copy()
        df_copia['diff'] = abs(df_copia['order_date'] - linea_venta['invoice_date'])
        df_sort = df_copia.sort_values('diff').reset_index()
        df_sort['cumsum'] = df_sort['product_qty'].cumsum()
        index = df_sort[df_sort['cumsum'] == linea_venta['quantity']].index[0] if not df_sort[df_sort['cumsum'] == linea_venta['quantity']].index.empty else None
        
        if index != None:
            df_sort.loc[:index , 'fact_line_id'] = linea_venta['fact_line_id']
            
            #Esta línea es para poder detectar la línea de compra con el costo mayor y dejarla para la línea de venta
            df_to_keep = (
                    df_sort.loc[
                            ~df_sort['fact_line_id'].isna()
                        ]
                        .sort_values('product_cost', ascending=False)
                        .reset_index()
                    )

            if match_3.empty:
                match_3 = df_to_keep.loc[:0, ['line_id', 'fact_line_id']].copy()
                match_3_to_errase = df_to_keep[['line_id', 'fact_line_id']].copy()

            else:
                match_3 = pd.concat([
                            match_3, 
                            df_to_keep.loc[:0, ['line_id', 'fact_line_id']],
                        ])
                match_3_to_errase = pd.concat([
                            match_3_to_errase, 
                            df_to_keep[['line_id', 'fact_line_id']],
                        ])

In [ ]:
# Después de correr el 3er match, las ventas restantes
ventas_after_match_3 = ventas_after_match_2.loc[
                                ~ventas_after_match_2['fact_line_id'].isin(match_3_to_errase['fact_line_id'])
                            ]

# Después de correr el 3er match, las compras que restan y que sí tendrán venta a la cual mercharse.
compras_especiales_after_match_3 = compras_especiales_after_match_2.loc[
                                            (~compras_especiales_after_match_2['line_id'].isin(match_3_to_errase['line_id']))
                                            & (compras_especiales_after_match_2['product_id_pp'].isin(ventas_after_match_3['product_id']))
                                        ]

# Después de correr el 3er match, resto de las compras especiales. Estan no tienen un product_id que existe en las ventas restantes y no se pueden merchar.
compras_especiales_after_match_3_sin_linea_venta = compras_especiales_after_match_2.loc[
                                                            (~compras_especiales_after_match_2['line_id'].isin(match_3_to_errase['line_id']))
                                                            & (~compras_especiales_after_match_2['product_id_pp'].isin(ventas_after_match_3['product_id']))
                                                        ]

In [ ]:
match_4_merge = (
    pd.merge_asof(
        left = ventas_after_match_3.sort_values('invoice_date'), 
        right = compras_especiales_after_match_3.sort_values('order_date'),
        
        left_by = ['product_id'], 
        right_by = ['product_id_pp'], 
        
        left_on = 'invoice_date', 
        right_on = 'order_date', 

        direction = 'nearest',
    )
)

In [ ]:
# Match resultantes del 4to match.
match_4 = match_4_merge.loc[~match_4_merge['line_id'].isna(), ['line_id', 'fact_line_id']]

# Después de correr el 1er match, las ventas restantes
ventas_after_match_4 = ventas_after_match_3.loc[
                                ~ventas_after_match_3['fact_line_id'].isin(match_4['fact_line_id'])
                            ]

# Después de correr el 4to match, las compras que tienen un product_id que sí existe en las ventas restantes.
compras_especiales_after_match_4 = compras_especiales_after_match_3.loc[
                                            (~compras_especiales_after_match_3['line_id'].isin(match_1['line_id']))
                                            & (compras_especiales_after_match_3['product_id_pp'].isin(ventas_after_match_4['product_id']))
                                        ]

# Después de correr el 4to match, resto de las compras especiales. Estan no tienen un product_id que existe en las ventas restantes y no se pueden merchar.
compras_especiales_after_match_4_sin_linea_venta = compras_especiales_after_match_3.loc[
                                                            (~compras_especiales_after_match_3['line_id'].isin(match_1['line_id']))
                                                            & (~compras_especiales_after_match_3['product_id_pp'].isin(ventas_after_match_4['product_id']))
                                                        ]

In [ ]:
match_compras_especiales = pd.concat(
                                [
                                    match_1_repetidos if not match_1_repetidos.empty else None, 
                                    match_1, 
                                    match_2, 
                                    match_3, 
                                    match_4
                                ]
                            )

In [ ]:
costo_venta_a = (
    match_compras_especiales.merge(
        ventas,
        how='left',
        on='fact_line_id'
    ).merge(
        compras_odoo,
        how='left',
        on='line_id'
    )
)

In [ ]:
costo_venta_b = (
    pd.merge_asof(
        left = ventas_after_match_4.sort_values('invoice_date'),
        right = pd.concat([
                        compras_no_especiales, 
                        ultimo_costo,
                ]).sort_values('order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'order_date', 

        direction = 'backward')
)

In [ ]:
cols_costo_venta =[
        # Documento de factura
        'name', 'invoice_date', 'partner_name_x', 
        # Vendedora
        'salesperson_id', 'salesperson_name',
        # Producto
        'product_name_x',
        # Precio de la partida
        'quantity', 'price_unit', 'discount', 'price_subtotal',
        # Documento de compra
        'order_name', 'order_date', 'partner_name_y',
        # Costo de la partida
        'product_cost', 'cost_subtotal',
        # Utilidad
        'utilidad_partida_$', 'utilidad_%', 'margen_contribución_%'
    ]

In [ ]:
costo_venta = pd.concat([costo_venta_a, costo_venta_b])

costo_venta['fact_line_id'] = costo_venta['fact_line_id'].astype('Int64')
costo_venta['line_id'] = costo_venta['line_id'].astype('Int64')

costo_venta['cost_subtotal'] = costo_venta['product_cost'] * costo_venta['quantity']
costo_venta['utilidad_partida_$'] = costo_venta['price_subtotal'] - costo_venta['cost_subtotal']
costo_venta['utilidad_%'] = ((costo_venta['price_subtotal'] / (costo_venta['cost_subtotal']) - 1) * 100).round(2)
costo_venta['margen_contribución_%'] = (costo_venta['utilidad_partida_$'] / costo_venta['price_subtotal'] * 100).round(2)

# Margen y utilidades

In [ ]:
wep = costo_venta[['salesperson_name', 'price_subtotal', 'cost_subtotal', 'utilidad_partida_$']].groupby('salesperson_name', dropna=False).sum()

wep['utilidad_%'] = ((wep['price_subtotal'] / (wep['cost_subtotal']) - 1) * 100).round(2)
wep['margen_contribución_%'] = (wep['utilidad_partida_$'] / wep['price_subtotal'] * 100).round(2)

wep

In [ ]:
wep_mes = wep.agg({'price_subtotal':['sum'], 'cost_subtotal':['sum'], 'utilidad_partida_$':['sum']})

wep_mes['utilidad_%'] = ((wep_mes['price_subtotal'] / (wep_mes['cost_subtotal']) - 1) * 100).round(2)
wep_mes['margen_contribución_%'] = (wep_mes['utilidad_partida_$'] / wep_mes['price_subtotal'] * 100).round(2)

wep_mes

# Checks

In [ ]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(pd.concat([prov_locales, prov_oficiales])['partner_id'])]).drop_duplicates('partner_id')

if check1.empty:
    print('Check 1 correcto')

else:
    print('Check 1 con error: hay proveedores no calificados en la lista de proeveedores oficiales.')
    display(check1)

In [ ]:
# Check donde se verifíca que ya no hay más compras especiales por merchar, es decir, ya no se ocupa un match_5.
check2 = compras_especiales_after_match_4.empty

if check2:
    print('Check 2 correcto')

else:
    print('Check 2 con error: hay compras especiales remanentes después del último match.')

In [ ]:
# Check donde todas las líneas compras especiales están unidas a líneas de ventas
all_line_ids_processed = (
        pd.concat(
            [
                match_compras_especiales['line_id'],
                match_3_to_errase.loc[~match_3_to_errase['line_id'].isin(match_3), 'line_id'],
                compras_especiales_after_match_1_sin_linea_venta['line_id'],
                compras_especiales_after_match_2_sin_linea_venta['line_id'],
                compras_especiales_after_match_3_sin_linea_venta['line_id'],
                compras_especiales_after_match_4_sin_linea_venta['line_id'],
            ]
        )
        .drop_duplicates()
        .sort_values()
        .astype('Int64')
        .reset_index()
        ['line_id']
    )

all_line_ids_to_process = (
        compras_especiales['line_id']
        .sort_values()
        .reset_index()
        ['line_id']
    )

check3 = all_line_ids_processed.equals(all_line_ids_to_process)

if check3:
    print('Check 3 correcto')

else:
    print('Check 3 con error: Hay líneas de compra especiales que aun no se han procesado.')

In [ ]:
# Check para ver el tamaño del costo de ventas generado vs las ventas del mes
check4 = len(costo_venta) == len(ventas)

if check4:
    print('Check 4 correcto')

else:
    print('Check 4 con error: el tamaño del costo de ventas es diferente al tamaño de las ventas iniciales.')

In [ ]:
# Check para ver que no existen lineas de venta fuera del costo de ventas.
check5 = ventas[~ventas['fact_line_id'].isin(costo_venta['fact_line_id'])].empty

if check5:
    print('Check 5 correcto')

else:
    print('Check 5 con error: el costo de ventas no contempla unas líneas de venta del dataframe de ventas iniciales.')

In [ ]:
# Check para que todas las líneas de venta del costo de ventas tienen un costo.
check6 = costo_venta.loc[
        costo_venta['order_name'].isna(),
        ['product_id', 'product_name_x']
    ].groupby('product_id').first()

if check6.empty:
    print('Check 6 correcto')

else:
    print('Check 6 con error: hay productos sin costo de ventas. Son los siguientes:')
    display(check6)

In [ ]:
check7 = costo_venta.loc[
        (costo_venta['move_type'] != 'out_refund')
        & (costo_venta['quantity'] != 0)
        & (costo_venta['utilidad_partida_$'] <= 0)
    ][cols_costo_venta].sort_values('utilidad_partida_$')

if check7.empty:
    print('Check 7 correcto')

else:
    print('Check 7 con error: Hay líneas de venta con utilidades negativas o en cero. Son las siguientes:')
    display(check7)